# Mutli-Probe LSH, Lv et al

[Paper here](https://www.cs.princeton.edu/cass/papers/mplsh_vldb07.pdf)


## 1. Introduction

- Classic LSH method needs to use multiple hash tables to produce a good candidate set.
- Experiments suggest >= 100 hash tables for "good search accuracy for high-dimensional datasets"
- Panigrahy proposed entropy-based LSH to generate randomly perturbed objects near the query object, query them in addition to the query object, and return the union of all results as the candidate set.
- Main ideas of this paper:

> build on the basic LSH indexing method, but use a carefully derived probing sequence to look up multiple buckets with a high probability of containing the nearest neighbors.

> by probing multiple buckets in each hash table, the method requires fewer hash tables than previously proposed LSH methods.


## 3. LSH

### 3.3 Entropy-Based LSH Indexing

- Assume you know the distance $R_p$ from the a query point $q$ to its nearest neighbor $p$. Not a practical assumption IMO.
- Generate some random points at distance $R_p$ from $q$, hash them, and accumulate the union of buckets for the hashes.
- Lv, et. al. report that this decreases the number of hash tables by a factor of 2 to 3, but requires 30% - 210% longer queries. 
- It's also impractical to assume you know the distance $R_p$ in a data-independent way. I guess you could sample some vectors, compute exact distances, and use those for $R_p$.
- You also get many duplicate hashes/buckets from the perturbed vectors, meaning much of the computation is wasted.

## 4. Multi-Probe LSH Indexing

### 4.1 Algorithm Overview

Goal is to locate the nearby buckets.

Aside: would it be possible to maintain the centroid of each bucket? This makes sense for euclidean distances but not really for non-euclidean.

Hash perturbation vector is a vector $\Delta = (\delta_1, ... \delta_M)$, where $M$ is the number of LSH functions for a table.

Given a query $q$, basic LSH checks the hash bucket $g(q) = (h_1(q), ... h_M(q))$.

When applying the perturbation vector $\Delta$, we check the hash bucket $g(q) + \Delta$.

For the $l_2$ distance LSH function $h_{a,b}(v) = \lfloor \frac{a \dot v + b}{W} \rfloor$, they assume that similar objects should hash to the same or adjacent values. Hence, restrict to perturbation vectors $\Delta$ with $\delta_i \in {-1, 0, 1}$. (I'm not sure if this assumption holds for the other distance functions...)

Each perturbation vector gets applied to the _hash_ values of the query object, not the query object itself. This avoids the overhead of point perturbation and re-hashing from the entropy LSH.

### 4.2 Step-Wise Probing

$n$-step perturbation vector $\Delta$ has $n$ non-zero coordinates.

The step-wise probing method first probes all 1-step buckets, then all 2-step buckets, and so on.

For an LSH index with $L$ hash tables and $M$ hash functions pwer table, the total number of $n$-step buckets is $L \times {M \choose n} \times 2^n$
 
  - $L$ is the number of tables
  - $M \choose n$ is the number of ways you can perturb $n$ hash values
  - $2^n$ is the number of non-zero positions, each with possible values -1 and +1
  
Important properties:

- All coordinates in the hash values of the query $q$ have equal probability of being perturbed.
- It's equally likely to add 1 and subtract 1 from any perturbed coordinate.

### 4.3 Success Probability Estimation

A more refined probing sequence is possible by considering how the hash function is computed.

Each hash function $h_{a,b}(q) = \lfloor \frac{a \dot q + b}{W} \rfloor$ first maps $q$ onto a line. The probability that a point $p$ falls into the slot to the "right" or "left" of $q$ depends on how close $q$ is to the right or left boundary of its slot. So the position of $q$ within its slot is potentially useful in determining which perturbations to pursue.

$f_i(q) = a_i \dot q + b_i$. This is the projection of query $q$ onto the line for the $i$-th hash function.

$h_i(q) = \lfloor \frac{a_i \dot q + b_i}{W} \rfloor$, this is the $i$-th hash function and equivalently the "slot" to which $q$ is hashed.

For $\delta \in {-1, + 1}$, let $x_i(\delta)$ be the distance of $q$ from the boundary of the slot $h_i(q) + \delta$.

$x_i(-1) = f_i(q) - h_i(q) \times W$

$x_i(1) = W - x_i(-1)$

$x_i(0) = 0$ for convenience.

...a bunch of math...

$\text{score}(\Delta) = \sum_{i=1}^M x_i(\delta_i)^2$

Perturbation vectors with smaller scores have higher probabilities of yield points near to $q$.


### 4.4 Query-Directed Probing Sequence

It would be slow to explicitly generate all perturbation vectors and sort them by scores, especially since only a small fraction will be used.

Perturbation vector scores depend only on non-zero coordinates of $\Delta$, and we expect that perturbation vectors with low scores will have few non-zero coordinates.

Represent the non-zero coordinates as a set of $(i, \delta_i)$ tuples, where $(i, \delta)$ represents adding $\delta$ to the $i$-th value of $q$.

(Need to come back and review this section more thoroughly. It's intuitive but very dense.)



***

**Elastiknn implications**

- Should try query-directed multi-probe at query-time.
- Still requires storing multiple tables, so this should build on top of the standard LSH implementation.
- Need to figure out how this applies to distance functions other than $l_2$, which was the only one presented in this paper. After some searching, I don't see anything related to Jaccard similarity. It kind of makes sense that there wouldn't be an analagous method for Jaccard similarity. With the $l_2$ hashing scheme, it's reasonable to assume that consecutive hash values are spatially related. I can't really draw a parallel with binary similarity like Jaccard, where you'd have to express nearness along a random permutation.














